<a href="https://colab.research.google.com/github/DENISOV-SERGEI/Multi-model-AI-for-efficient-resource-management/blob/main/PE9_1_multi_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Пишем код для отправки запросов, конфигурируем модели

In [1]:
!pip install openai >> None

In [2]:
import openai
openai.__version__

'1.109.1'

In [3]:
from openai import OpenAI
import time

from google.colab import userdata
API_KEY = userdata.get('OpenAI_API_key')

client = OpenAI(api_key=API_KEY)

config = {'intent':{'model':"gpt-3.5-turbo-0125",
                            'system_prompt':"""You are a helpful assistant. Classify the following prompt as a 'simple' or 'hard' task.
             The task is 'hard' only if it requires the background in computer science. Return just one lower-case word 'simple' or 'hard':""",
                    'max_tokens':5,
                    'price_input':0.5,
                    'price_output':1.5},

                  'easy':{'model':"gpt-3.5-turbo-0125",
                           'system_prompt':"You're a helpful assistant",
                           'max_tokens':None,
                           'price_input':0.5,
                           'price_output':1.5},
                  'hard':{'model':"gpt-4o",
                           'system_prompt':"You're a helpful assistant",
                           'max_tokens':None,
                           'price_input':5,
                           'price_output':15}}

def call_model(prompt, model_type):
    start_time = time.time()
    response = client.chat.completions.create(
        model=config[model_type]['model'],
        messages=[
            {"role": "system",
             "content": config[model_type]['system_prompt']},
            {"role": "user", "content": prompt}
        ],
        max_tokens=config[model_type]['max_tokens']
    )
    latency = time.time() - start_time

    if model_type == 'intent':
        intent = response.choices[0].message.content
    else:
        intent = None
    tokens_input = response.usage.prompt_tokens
    tokens_output = response.usage.completion_tokens
    price = tokens_input*config[model_type]['price_input'] + tokens_output*config[model_type]['price_output']
    print(response.choices[0].message.content)
    return price, latency, intent

# Задаем тестовые вопросы, тестируем как с ними справляется классификатор

In [4]:
EASY_QUERY = 'Give me a recipe for a classic margarita cocktail.'
HARD_QUERY = 'How do memory-mapped files (mmap) provide advantages over traditional read/write system calls for certain types of I/O operations?'

call_model(HARD_QUERY, 'intent')
call_model(EASY_QUERY, 'intent')

hard
simple


(39.0, 0.983374834060669, 'simple')

# Используем только мощную модель

In [5]:
price_1,latency_1, _ = call_model(HARD_QUERY, 'hard')
price_2,latency_2, _ = call_model(EASY_QUERY, 'hard')
tot_price = price_1 + price_2
tot_latency = latency_1 + latency_2
print('\n-------------------------------\nTOTAL PRICE IS {}, LATENCY {}'.format(tot_price, tot_latency))

Memory-mapped files (mmap) offer several advantages over traditional read/write system calls for certain types of I/O operations, primarily due to their ability to provide a more direct and efficient way of accessing file contents. Here's how they offer advantages:

1. **Efficiency and Performance:**
   - **Reduced System Call Overhead:** With memory-mapped files, data is accessed through memory rather than through explicit read and write system calls. This reduces the overhead associated with multiple I/O system calls, as accessing memory is generally faster than invoking kernel routines.
   - **Automatic Caching:** The operating system automatically caches data in memory when using mmap, which can lead to performance improvements. If the required data is already loaded in memory, subsequent accesses are much faster.
   - **Zero-Copy I/O:** Data is not copied between user-space and kernel-space, as it typically is with read/write. With mmap, file data is mapped directly into the proce

# Используем мульти-модельность

In [6]:
def router(QUERY):
    price_intent,latency_intent, intent = call_model(QUERY, 'intent')
    if intent.strip().lower() == 'hard':
        price_answer, latency_answer, _ = call_model(QUERY, 'hard')
    else:
        price_answer, latency_answer, _ = call_model(QUERY, 'easy')

    total_price = price_intent + price_answer
    total_latency = latency_intent + latency_answer
    return total_price, total_latency

In [7]:
price_1,latency_1 = router(HARD_QUERY)
price_2,latency_2 = router(EASY_QUERY)
tot_price = price_1 + price_2
tot_latency = latency_1 + latency_2
print('\n-------------------------------\nTOTAL PRICE IS {}, LATENCY {}'.format(tot_price, tot_latency))

hard
Memory-mapped files (mmap) offer several advantages over traditional read/write system calls, especially for certain types of I/O operations that can benefit from more efficient memory and data access patterns. Here are some of the key benefits:

1. **Performance Improvement**:  
   - **Reduced System Call Overhead**: Once a file is memory-mapped, data can be accessed directly in memory rather than through read or write system calls, which can significantly reduce overhead caused by these frequent calls.
   - **Efficient Read/Write**: Memory-mapped files allow random access to data directly through memory reads and writes, which can be faster than performing multiple I/O operations with traditional methods. This is particularly beneficial for applications that need to access or modify data at irregular offsets.

2. **Zero-Copy I/O**:
   - Memory mapping can eliminate the need to copy data between kernel space and user space, which is a requirement in traditional I/O approaches. Wh